In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn as skl
from tqdm.notebook import tqdm
import os
from google.cloud import bigquery

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '...' 
os.environ['GCLOUD_PROJECT'] = '...'
%load_ext google.cloud.bigquery

In [ ]:
# Import required library
from google.cloud import bigquery

# Initialize client
client = bigquery.Client()

# List datasets in project
datasets = list(client.list_datasets())

In [ ]:
client=bigquery.Client()
project = client.project
dataset_ref = client.dataset('...', project='...')
tables = client.list_tables('...')
for i in tables:
    print(i.table_id)

## Pull pre-training cohort

In [ ]:
%%time
QUERY= """
WITH antihypertensive_cohort AS (
    SELECT DISTINCT de.person_id, 
           MIN(de.drug_exposure_start_DATETIME) as first_exposure_date
    FROM `drug_exposure` de
    INNER JOIN `concept_ancestor` ca 
        ON de.drug_concept_id = ca.descendant_concept_id
    WHERE ca.ancestor_concept_id IN (
        21600381, 21601783, 21601744, 21601853, 21601796
    )
    GROUP BY de.person_id
)

SELECT 
    ac.person_id,
    p.birth_datetime
FROM antihypertensive_cohort ac
JOIN `person` p
    ON ac.person_id = p.person_id
ORDER BY ac.person_id;
"""
query_job =client.query(QUERY)
dob=query_job.to_dataframe()
dob.head()

In [ ]:
dob.to_csv('./raw_data/dob_antihypertensives.csv')

In [ ]:
%%time
QUERY= """
WITH antihypertensive_cohort AS (
    SELECT DISTINCT de.person_id, 
           MIN(de.drug_exposure_start_DATETIME) as first_exposure_date
    FROM `drug_exposure` de
    INNER JOIN `concept_ancestor` ca 
        ON de.drug_concept_id = ca.descendant_concept_id
    WHERE ca.ancestor_concept_id IN (
        21600381, 21601783, 21601744, 21601853, 21601796
    )
    GROUP BY de.person_id
),

all_events AS (
    -- Drugs
    SELECT 
        ac.person_id,
        'drug' as domain,
        de.drug_concept_id as concept_id,
        de.drug_exposure_start_DATETIME as event_date,
        c.concept_name
    FROM antihypertensive_cohort ac
    JOIN `drug_exposure` de 
        ON ac.person_id = de.person_id
    JOIN `concept` c 
        ON de.drug_concept_id = c.concept_id
    WHERE de.drug_exposure_start_DATETIME < ac.first_exposure_date

    UNION ALL

    -- Conditions
    SELECT 
        ac.person_id,
        'condition' as domain,
        co.condition_concept_id as concept_id,
        co.condition_start_datetime as event_date,
        c.concept_name
    FROM antihypertensive_cohort ac
    JOIN `condition_occurrence` co 
        ON ac.person_id = co.person_id
    JOIN `concept` c 
        ON co.condition_concept_id = c.concept_id
    WHERE co.condition_start_datetime < ac.first_exposure_date

    UNION ALL

    -- Procedures
    SELECT 
        ac.person_id,
        'procedure' as domain,
        po.procedure_concept_id as concept_id,
        po.procedure_DATETIME as event_date,
        c.concept_name
    FROM antihypertensive_cohort ac
    JOIN `procedure_occurrence` po 
        ON ac.person_id = po.person_id
    JOIN `concept` c 
        ON po.procedure_concept_id = c.concept_id
    WHERE po.procedure_DATETIME < ac.first_exposure_date

    UNION ALL

    -- Observations
    SELECT 
        ac.person_id,
        'observation' as domain,
        o.observation_concept_id as concept_id,
        o.observation_datetime as event_date,
        c.concept_name
    FROM antihypertensive_cohort ac
    JOIN `observation` o 
        ON ac.person_id = o.person_id
    JOIN `concept` c 
        ON o.observation_concept_id = c.concept_id
    WHERE o.observation_datetime < ac.first_exposure_date
    AND o.observation_concept_id != 2000006253 --these are unknown flowsheet entries with no meaningful detail so we exclude

    UNION ALL

    -- Measurements
    SELECT 
        ac.person_id,
        'measurement' as domain,
        m.measurement_concept_id as concept_id,
        m.measurement_DATETIME as event_date,
        c.concept_name
    FROM antihypertensive_cohort ac
    JOIN `measurement` m 
        ON ac.person_id = m.person_id
    JOIN `concept` c 
        ON m.measurement_concept_id = c.concept_id
    WHERE m.measurement_DATETIME < ac.first_exposure_date
),
patient_event_counts AS (
    SELECT person_id, COUNT(*) as event_count
    FROM all_events
    GROUP BY person_id
    HAVING COUNT(*) >= 10
),

filtered_events AS (
    SELECT DISTINCT person_id, domain, concept_id, event_date, concept_name
    FROM all_events
    WHERE concept_id != 0
),

ranked_events AS (
    SELECT 
        e.*,
        ROW_NUMBER() OVER(
            PARTITION BY e.person_id 
            ORDER BY e.event_date DESC
        ) as event_rank
    FROM filtered_events e
    INNER JOIN patient_event_counts pec
        ON e.person_id = pec.person_id
)

SELECT 
    person_id,
    domain,
    concept_id,
    event_date,
    concept_name
FROM ranked_events
WHERE event_rank <= 1024 --if you remove this line I think you will get all data
ORDER BY person_id, event_date DESC;
"""

QUERY = """
WITH antihypertensive_cohort AS (
    SELECT DISTINCT de.person_id, 
           MIN(de.drug_exposure_start_DATETIME) as first_exposure_date
    FROM `drug_exposure` de
    INNER JOIN `concept_ancestor` ca 
        ON de.drug_concept_id = ca.descendant_concept_id
    WHERE ca.ancestor_concept_id IN (
        21600381, 21601783, 21601744, 21601853, 21601796
    )
    GROUP BY de.person_id
),

all_events AS (
    -- Drugs
    SELECT 
        ac.person_id,
        'drug' as domain,
        de.drug_concept_id as concept_id,
        de.drug_exposure_start_DATETIME as event_date,
        c.concept_name
    FROM antihypertensive_cohort ac
    JOIN `drug_exposure` de 
        ON ac.person_id = de.person_id
    JOIN `concept` c 
        ON de.drug_concept_id = c.concept_id
    WHERE de.drug_exposure_start_DATETIME < ac.first_exposure_date

    UNION ALL

    -- Conditions
    SELECT 
        ac.person_id,
        'condition' as domain,
        co.condition_concept_id as concept_id,
        co.condition_start_datetime as event_date,
        c.concept_name
    FROM antihypertensive_cohort ac
    JOIN `condition_occurrence` co 
        ON ac.person_id = co.person_id
    JOIN `concept` c 
        ON co.condition_concept_id = c.concept_id
    WHERE co.condition_start_datetime < ac.first_exposure_date

    UNION ALL

    -- Procedures
    SELECT 
        ac.person_id,
        'procedure' as domain,
        po.procedure_concept_id as concept_id,
        po.procedure_DATETIME as event_date,
        c.concept_name
    FROM antihypertensive_cohort ac
    JOIN `procedure_occurrence` po 
        ON ac.person_id = po.person_id
    JOIN `concept` c 
        ON po.procedure_concept_id = c.concept_id
    WHERE po.procedure_DATETIME < ac.first_exposure_date

    UNION ALL

    -- Observations
    SELECT 
        ac.person_id,
        'observation' as domain,
        o.observation_concept_id as concept_id,
        o.observation_datetime as event_date,
        c.concept_name
    FROM antihypertensive_cohort ac
    JOIN `observation` o 
        ON ac.person_id = o.person_id
    JOIN `concept` c 
        ON o.observation_concept_id = c.concept_id
    WHERE o.observation_datetime < ac.first_exposure_date
    AND o.observation_concept_id != 2000006253 --these are unknown flowsheet entries with no meaningful detail so we exclude

    UNION ALL

    -- Measurements
    SELECT 
        ac.person_id,
        'measurement' as domain,
        m.measurement_concept_id as concept_id,
        m.measurement_DATETIME as event_date,
        c.concept_name
    FROM antihypertensive_cohort ac
    JOIN `measurement` m 
        ON ac.person_id = m.person_id
    JOIN `concept` c 
        ON m.measurement_concept_id = c.concept_id
    WHERE m.measurement_DATETIME < ac.first_exposure_date
),

patient_event_counts AS (
    SELECT 
        person_id, 
        COUNT(*) as event_count
    FROM all_events
    WHERE concept_id != 0
    GROUP BY person_id
    HAVING COUNT(*) >= 10
),

-- Statistics calculation
patient_stats AS (
    SELECT 
        COUNT(*) as total_patients,
        COUNTIF(event_count > 1024) as patients_exceeding_limit,
        COUNTIF(event_count > 1024) / COUNT(*) as proportion_exceeding
    FROM patient_event_counts
)

-- Return the statistics
SELECT 
    total_patients,
    patients_exceeding_limit,
    proportion_exceeding,
    proportion_exceeding * 100 as percentage_exceeding
FROM patient_stats;
"""
query_job =client.query(QUERY)
data=query_job.to_dataframe()
data.head()

In [ ]:
data['person_id'].nunique()

In [ ]:
%%time
data.to_csv('./raw_data/antihypertensive_1024_events_prior_to_med_start.csv')

In [ ]:
%%time
QUERY= """
SELECT DISTINCT 
    de.person_id,
    MIN(de.drug_exposure_start_DATETIME) as first_antihypertensive_date
FROM `drug_exposure` de
INNER JOIN `concept_ancestor` ca 
    ON de.drug_concept_id = ca.descendant_concept_id
WHERE ca.ancestor_concept_id IN (
    21600381, -- Drugs used in the treatment of hypertension
    21601783, -- ACE inhibitors
    21601744, -- Beta blocking agents
    21601853, -- Calcium channel blockers
    21601796  -- Agents acting on the renin-angiotensin system
)
GROUP BY de.person_id
ORDER BY de.person_id;
"""
query_job =client.query(QUERY)
start_date=query_job.to_dataframe()
start_date.head()

In [ ]:
start_date.to_csv('./raw_data/antihypertensive_start.csv')

In [ ]:
%%time
QUERY= """
WITH antihypertensive_cohort AS (
    SELECT DISTINCT de.person_id, 
           MIN(de.drug_exposure_start_DATETIME) as first_exposure_date
    FROM `drug_exposure` de
    INNER JOIN `concept_ancestor` ca 
        ON de.drug_concept_id = ca.descendant_concept_id
    WHERE ca.ancestor_concept_id IN (
        21600381, 21601783, 21601744, 21601853, 21601796
    )
    GROUP BY de.person_id
),

first_mace AS (
    SELECT 
        ac.person_id,
        MIN(c.condition_start_date) as first_mace_date
    FROM antihypertensive_cohort ac
    LEFT JOIN `condition_occurrence` c 
        ON ac.person_id = c.person_id --VALIDATE THIS LIST
        AND (c.condition_source_value LIKE '410.%'      -- Acute MI
        OR c.condition_source_value LIKE '411.%'         -- Other acute/subacute IHD
        OR c.condition_source_value LIKE '412.%'         -- Old MI
        OR c.condition_source_value LIKE '413.%'         -- Angina pectoris
        OR c.condition_source_value LIKE '414.%'         -- Chronic IHD
        OR c.condition_source_value LIKE '428.%'         -- Heart failure
        OR c.condition_source_value LIKE '430.%'         -- Subarachnoid hemorrhage
        OR c.condition_source_value LIKE '431.%'         -- Intracerebral hemorrhage
        OR c.condition_source_value LIKE '432.%'         -- Other intracranial hemorrhage
        OR c.condition_source_value LIKE '433.%'         -- Occlusion/stenosis precerebral
        OR c.condition_source_value LIKE '434.%'         -- Occlusion cerebral arteries
        OR c.condition_source_value LIKE '435.%'         -- TIA
        OR c.condition_source_value LIKE '436.%'         -- Acute CVA
        OR c.condition_source_value LIKE '437.%'         -- Other cerebrovascular disease
        OR c.condition_source_value LIKE 'I21.%'         -- Acute MI
        OR c.condition_source_value LIKE 'I22.%'         -- Subsequent MI
        OR c.condition_source_value LIKE 'I23.%'         -- Complications following MI
        OR c.condition_source_value LIKE 'I24.%'         -- Other acute IHD
        OR c.condition_source_value LIKE 'I50.%'         -- Heart failure
        OR c.condition_source_value LIKE 'I63.%'         -- Cerebral infarction
        OR c.condition_source_value LIKE 'I65.%'         -- Occlusion/stenosis precerebral
        OR c.condition_source_value LIKE 'I66.%')         -- Occlusion cerebral arteries
        AND c.condition_start_date > ac.first_exposure_date
    GROUP BY ac.person_id
)

SELECT 
    ac.person_id,
    ac.first_exposure_date,
    CAST(COALESCE(m.first_mace_date, NULL) AS STRING) as first_mace_date
FROM antihypertensive_cohort ac
LEFT JOIN first_mace m ON ac.person_id = m.person_id
ORDER BY ac.person_id;
"""
query_job =client.query(QUERY)
first_mace=query_job.to_dataframe()
first_mace.head()

In [ ]:
first_mace.to_csv('./raw_data/first_mace_post_antihypertensives.csv')

In [ ]:
%%time
QUERY= """
WITH antihypertensive_cohort AS (
    SELECT DISTINCT de.person_id, 
           MIN(de.drug_exposure_start_DATETIME) as first_exposure_date
    FROM `drug_exposure` de
    INNER JOIN `concept_ancestor` ca 
        ON de.drug_concept_id = ca.descendant_concept_id
    WHERE ca.ancestor_concept_id IN (
        21600381, 21601783, 21601744, 21601853, 21601796
    )
    GROUP BY de.person_id
),

death_dates AS (
    SELECT 
        ac.person_id,
        d.death_date
    FROM antihypertensive_cohort ac
    LEFT JOIN `death` d 
        ON ac.person_id = d.person_id
        AND d.death_date > ac.first_exposure_date
)

SELECT 
    ac.person_id,
    ac.first_exposure_date,
    CAST(COALESCE(d.death_date, NULL) AS STRING) as death_date
FROM antihypertensive_cohort ac
LEFT JOIN death_dates d ON ac.person_id = d.person_id
ORDER BY ac.person_id;
"""
query_job =client.query(QUERY)
death=query_job.to_dataframe()
death.head()

In [ ]:
death.to_csv('./raw_data/death.csv')

## Pull fine tuning + eval cohort

In [ ]:
#pulls most recent 1024 events (before dox) for patients on doxorubicin
QUERY = """
WITH doxorubicin_concepts AS (
    SELECT concept_id
    FROM `concept`
    WHERE LOWER(concept_name) LIKE '%doxorubicin%'
    AND LOWER(concept_name) NOT LIKE '%doxycycline%'
),

doxorubicin_cohort AS (
    SELECT DISTINCT de.person_id, 
           MIN(de.drug_exposure_start_DATETIME) as first_exposure_date
    FROM `drug_exposure` de
    WHERE de.drug_concept_id IN (SELECT concept_id FROM doxorubicin_concepts)
    GROUP BY de.person_id
)

SELECT 
    dc.person_id,
    p.birth_datetime
FROM doxorubicin_cohort dc
JOIN `person` p
    ON dc.person_id = p.person_id
ORDER BY dc.person_id;
"""

In [ ]:
%%time
query_job =client.query(QUERY)
dob=query_job.to_dataframe()
print(dob.shape)
print(dob['person_id'].nunique())
dob.head()

In [ ]:
dob.to_csv('./raw_data/dox_dob.csv')

In [ ]:
#pulls most recent 1024 events (before dox) for patients on doxorubicin
QUERY = """
WITH doxorubicin_concepts AS (
    SELECT concept_id
    FROM `concept`
    WHERE LOWER(concept_name) LIKE '%doxorubicin%'
    AND LOWER(concept_name) NOT LIKE '%doxycycline%'
),

doxorubicin_cohort AS (
    SELECT DISTINCT de.person_id, 
           MIN(de.drug_exposure_start_DATETIME) as first_exposure_date
    FROM `drug_exposure` de
    WHERE de.drug_concept_id IN (SELECT concept_id FROM doxorubicin_concepts)
    GROUP BY de.person_id
),

all_events AS (
    -- Drugs
    SELECT 
        dc.person_id,
        'drug' as domain,
        de.drug_concept_id as concept_id,
        de.drug_exposure_start_DATETIME as event_date,
        c.concept_name
    FROM doxorubicin_cohort dc
    JOIN `drug_exposure` de 
        ON dc.person_id = de.person_id
    JOIN `concept` c 
        ON de.drug_concept_id = c.concept_id
    WHERE de.drug_exposure_start_DATETIME < dc.first_exposure_date

    UNION ALL

    -- Conditions
    SELECT 
        dc.person_id,
        'condition' as domain,
        co.condition_concept_id as concept_id,
        co.condition_start_datetime as event_date,
        c.concept_name
    FROM doxorubicin_cohort dc
    JOIN `condition_occurrence` co 
        ON dc.person_id = co.person_id
    JOIN `concept` c 
        ON co.condition_concept_id = c.concept_id
    WHERE co.condition_start_datetime < dc.first_exposure_date

    UNION ALL

    -- Procedures
    SELECT 
        dc.person_id,
        'procedure' as domain,
        po.procedure_concept_id as concept_id,
        po.procedure_DATETIME as event_date,
        c.concept_name
    FROM doxorubicin_cohort dc
    JOIN `procedure_occurrence` po 
        ON dc.person_id = po.person_id
    JOIN `concept` c 
        ON po.procedure_concept_id = c.concept_id
    WHERE po.procedure_DATETIME < dc.first_exposure_date

    UNION ALL

    -- Observations
    SELECT 
        dc.person_id,
        'observation' as domain,
        o.observation_concept_id as concept_id,
        o.observation_datetime as event_date,
        c.concept_name
    FROM doxorubicin_cohort dc
    JOIN `observation` o 
        ON dc.person_id = o.person_id
    JOIN `concept` c 
        ON o.observation_concept_id = c.concept_id
    WHERE o.observation_datetime < dc.first_exposure_date
    AND o.observation_concept_id != 2000006253

    UNION ALL

    -- Measurements
    SELECT 
        dc.person_id,
        'measurement' as domain,
        m.measurement_concept_id as concept_id,
        m.measurement_DATETIME as event_date,
        c.concept_name
    FROM doxorubicin_cohort dc
    JOIN `measurement` m 
        ON dc.person_id = m.person_id
    JOIN `concept` c 
        ON m.measurement_concept_id = c.concept_id
    WHERE m.measurement_DATETIME < dc.first_exposure_date
),

patient_event_counts AS (
    SELECT person_id, COUNT(*) as event_count
    FROM all_events
    GROUP BY person_id
    HAVING COUNT(*) >= 10
),

filtered_events AS (
    SELECT DISTINCT person_id, domain, concept_id, event_date, concept_name
    FROM all_events
    WHERE concept_id != 0
),

ranked_events AS (
    SELECT 
        e.*,
        ROW_NUMBER() OVER(
            PARTITION BY e.person_id 
            ORDER BY e.event_date DESC
        ) as event_rank
    FROM filtered_events e
    INNER JOIN patient_event_counts pec
        ON e.person_id = pec.person_id
)

SELECT 
    person_id,
    domain,
    concept_id,
    event_date,
    concept_name
FROM ranked_events
WHERE event_rank <= 1024 --I think if you remove this line you will get all events before dox
ORDER BY person_id, event_date DESC;
"""

In [ ]:
%%time
query_job =client.query(QUERY)
dox_data=query_job.to_dataframe()
print(dox_data.shape)
print(dox_data['person_id'].nunique())
dox_data.head()

In [ ]:
%%time
query_job =client.query(QUERY)
dox_data=query_job.to_dataframe()
print(dox_data.shape)
print(dox_data['person_id'].nunique())
dox_data.head()

In [ ]:
%%time
dox_data.to_csv('./raw_data/dox_patients_1024_events_prior.csv')

In [ ]:
#pulls dox start date
QUERY = """
WITH doxorubicin_concepts AS (
    SELECT concept_id
    FROM `concept`
    WHERE LOWER(concept_name) LIKE '%doxorubicin%'
)

SELECT DISTINCT 
    de.person_id,
    MIN(de.drug_exposure_start_DATETIME) as first_doxorubicin_date
FROM `drug_exposure` de
WHERE de.drug_concept_id IN (SELECT concept_id FROM doxorubicin_concepts)
GROUP BY de.person_id
ORDER BY de.person_id;
"""

In [ ]:
%%time
query_job =client.query(QUERY)
start_date=query_job.to_dataframe()
start_date.head()

In [ ]:
start_date.to_csv('./raw_data/dox_start.csv')

In [ ]:
#pulls first MACE after dox exposure
QUERY = """
WITH doxorubicin_concepts AS (
    SELECT concept_id
    FROM `concept`
    WHERE LOWER(concept_name) LIKE '%doxorubicin%'
),

doxorubicin_cohort AS (
    SELECT DISTINCT de.person_id, 
           MIN(de.drug_exposure_start_DATETIME) as first_exposure_date
    FROM `drug_exposure` de
    WHERE de.drug_concept_id IN (SELECT concept_id FROM doxorubicin_concepts)
    GROUP BY de.person_id
),

first_mace AS (
    SELECT 
        dc.person_id,
        MIN(c.condition_start_date) as first_mace_date
    FROM doxorubicin_cohort dc
    LEFT JOIN `condition_occurrence` c 
        ON dc.person_id = c.person_id
        AND (c.condition_source_value LIKE '410.%'      -- Acute MI
        OR c.condition_source_value LIKE '411.%'         -- Other acute/subacute IHD
        OR c.condition_source_value LIKE '412.%'         -- Old MI
        OR c.condition_source_value LIKE '413.%'         -- Angina pectoris
        OR c.condition_source_value LIKE '414.%'         -- Chronic IHD
        OR c.condition_source_value LIKE '428.%'         -- Heart failure
        OR c.condition_source_value LIKE '430.%'         -- Subarachnoid hemorrhage
        OR c.condition_source_value LIKE '431.%'         -- Intracerebral hemorrhage
        OR c.condition_source_value LIKE '432.%'         -- Other intracranial hemorrhage
        OR c.condition_source_value LIKE '433.%'         -- Occlusion/stenosis precerebral
        OR c.condition_source_value LIKE '434.%'         -- Occlusion cerebral arteries
        OR c.condition_source_value LIKE '435.%'         -- TIA
        OR c.condition_source_value LIKE '436.%'         -- Acute CVA
        OR c.condition_source_value LIKE '437.%'         -- Other cerebrovascular disease
        OR c.condition_source_value LIKE 'I21.%'         -- Acute MI
        OR c.condition_source_value LIKE 'I22.%'         -- Subsequent MI
        OR c.condition_source_value LIKE 'I23.%'         -- Complications following MI
        OR c.condition_source_value LIKE 'I24.%'         -- Other acute IHD
        OR c.condition_source_value LIKE 'I50.%'         -- Heart failure
        OR c.condition_source_value LIKE 'I63.%'         -- Cerebral infarction
        OR c.condition_source_value LIKE 'I65.%'         -- Occlusion/stenosis precerebral
        OR c.condition_source_value LIKE 'I66.%')         -- Occlusion cerebral arteries
        AND c.condition_start_date > dc.first_exposure_date
    GROUP BY dc.person_id
)

SELECT 
    dc.person_id,
    dc.first_exposure_date,
    CAST(COALESCE(m.first_mace_date, NULL) AS STRING) as first_mace_date
FROM doxorubicin_cohort dc
LEFT JOIN first_mace m ON dc.person_id = m.person_id
ORDER BY dc.person_id;
"""

In [ ]:
%%time
query_job =client.query(QUERY)
first_mace=query_job.to_dataframe()
first_mace.head()

In [ ]:
first_mace.to_csv('./raw_data/first_mace_post_dox.csv')

In [ ]:
#pulls death data for dox patients
QUERY = """
WITH doxorubicin_concepts AS (
    SELECT concept_id
    FROM `concept`
    WHERE LOWER(concept_name) LIKE '%doxorubicin%'
),

doxorubicin_cohort AS (
    SELECT DISTINCT de.person_id, 
           MIN(de.drug_exposure_start_DATETIME) as first_exposure_date
    FROM `drug_exposure` de
    WHERE de.drug_concept_id IN (SELECT concept_id FROM doxorubicin_concepts)
    GROUP BY de.person_id
),

death_dates AS (
    SELECT 
        dc.person_id,
        d.death_date
    FROM doxorubicin_cohort dc
    LEFT JOIN `death` d 
        ON dc.person_id = d.person_id
        AND d.death_date > dc.first_exposure_date
)

SELECT 
    dc.person_id,
    dc.first_exposure_date,
    CAST(COALESCE(d.death_date, NULL) AS STRING) as death_date
FROM doxorubicin_cohort dc
LEFT JOIN death_dates d ON dc.person_id = d.person_id
ORDER BY dc.person_id;
"""

In [ ]:
query_job =client.query(QUERY)
death=query_job.to_dataframe()
death.head()

In [ ]:
death.to_csv('./raw_data/dox_death.csv')